# Homework \#6
Due August 5 @ 11:59pm

## Submission requirements
Upload a **single PDF file** of your IJulia notebook for this entire assigment. Clearly denote which question each section of your PDF corresponds to.

## Problem 1 -- Vehicle Routing

A local bakeshop wants determine the minimum cost plan for picking up its daily supply of milk, butter, and eggs from the four farms that supply the bakeshop.  The distance (in miles) between the bakeshop (B) and each farm, and also between each pair of farms, is given in the table below. The table also gives the volume of milk, butter, and eggs (total, in ft$^3$) to be collected from
each farm each day. (The distance between locations is the same in both directions, so for each pair of locations the distance is only reported once.)

| |B | F1 | F2| F3 | F4 |
|:--|:--|:--|:--|:--|:--|
|B| - | 5 | 12 | 7 | 15|
|F1| - | - | 4 | 10 | 7 |
|F2| - | - | - | 14 | 20|
|F3| - | - | - | - |8 |
| | F1 | F2 | F3 | F4 |
|Supply (ft$^3$)|  7 | 2 | 6 | 3 |

The bakeshop has one truck that can carry at most 10 ft$^3$ of supplies at a time. Because of the size limit, the
truck will need to make multiple trips each day to collect the supplies from all the farms. Each trip may pick up supplies from one or more farms, provided the total collected in the trip does not exceed the truck limit. 

Formulate an integer program to help the bakeshop assign farms to the trips so that the total number of trips required every day is minimized  (Hint: model the problem as a set covering problem. The first step will be to list all possible routes a truck can take.)

In [32]:
using JuMP, Gurobi, NamedArrays

# farms and possible routes
farms = [1,2,3,4]
route = [1,2,3,4,5]

# possible routes for one trip
trip = [1 1 0 0
        1 0 0 1
        0 1 1 0 
        0 1 0 1
        0 0 1 1]


# A is a matrix for distances
A=[5 12 7 15
   0 4 10 7
   0 0 14 20
   0 0 0 8]

m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m, "OutputFlag", 0)

# binary variable for whether we build a station in each city
@variable(m, x[route], Bin)

for col in farms
     @constraint(m, sum(trip[row,col]x[row] for row in route ) >= 1)
end

@objective(m, Min, sum(x))
                
optimize!(m)
x=value.(x)
display(x)
j=1
for r in route
    if x[r] == 1
        i=1
        for f in farms
            if trip[r,f]==1
                println("The ",i,"-th farm in trip",j," is F",f)
                i+=1
            end
            
        end
        j+=1
    end
    
end



1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [1, 2, 3, 4, 5]
And data, a 5-element Array{Float64,1}:
  0.0
  1.0
  1.0
 -0.0
 -0.0

Academic license - for non-commercial use only
Academic license - for non-commercial use only
The 1-th farm in trip1 is F1
The 2-th farm in trip1 is F4
The 1-th farm in trip2 is F2
The 2-th farm in trip2 is F3


## Problem 2 -- The Magical Baked Goods Machine

Suppose you are in charge of a magical baked goods machine that creates delicious baked goods of many varieties. Every day, the machine creates batches of 5 different baked goods. To produce a batch of a baked good, you must first clean the machine to remove the remnants of the previous batch of bakery treats (e.g., the workflow could be, "clean, make bread, clean, make donuts,..."). The durations of baking each of the 5 items (donuts, bread, cookies, scones, and coffee cake) are 40, 32, 50, 28, and 47 minutes respectively. The cleaning times depend on the item that was previously made in the machine. For example, a long cleaning period is required if bread is made after scones, since the scones leave a sticky residue from the dried fruit they contain that can ruin the bread. The times are given in minutes in the following table. Each pair $(i,j)$ is the cleaning time required if batch $j$ is baked after batch $i$.

|Baked good|Donut|Scone|Cookie|Bread|Coffee Cake|
|:---|:---|:----|:---|:---|:---|
|Donut|0|10|6|15|9|
|Scone|4|0|6|17|8|
|Cookie|10|11|0|20|14|
|Bread|7|15|6|0|2|
|Coffee Cake|5|8|7|7|0|

We'd obviously like to spend as little time as possible baking and cleaning. What order should we produce the 5 bakery items in? How long do we spend baking and cleaning each day? The order is the same every day, so the cleaning time between the last batch of one week and the first of the following week needs to be accounted for in the total duration of cleaning.

Solve this problem as a TSP. You may use either MTZ reformulation or subtour elimination (or both, for fun!).

In [25]:
using JuMP, NamedArrays, Gurobi

goods = [:Donut, :Scone, :Cookie, :Bread, :CoffeeCake]

times = [0 10 6 15 9
         4 0 6 17 8
         10 11 0 20 14
         7 15 6 0 2
         5 8 7 7 0]

c = NamedArray(times,(goods,goods),("from","to"))
N = size(times,1);
display(c)
println(N)

5×5 Named Array{Int64,2}
  from ╲ to │      :Donut       :Scone      :Cookie       :Bread  :CoffeeCake
────────────┼────────────────────────────────────────────────────────────────
:Donut      │           0           10            6           15            9
:Scone      │           4            0            6           17            8
:Cookie     │          10           11            0           20           14
:Bread      │           7           15            6            0            2
:CoffeeCake │           5            8            7            7            0

5


In [30]:
function getSubtour(x,start)
    subtour = [start]
    while true
        j = subtour[end]
        for k in goods
            if x[k,j] == 1
                push!(subtour,k)
                break
            end
        end
        if subtour[end] == start
            break
        end
    end
    return subtour
end

function getAllSubtours(x)
    nodesRemaining = goods
    subtours = []
    while length(nodesRemaining) > 0
        subtour = getSubtour(x,nodesRemaining[1])
        push!(subtours, subtour)
        nodesRemaining = setdiff(nodesRemaining,subtour)
    end
    return subtours
end

getAllSubtours (generic function with 1 method)

In [31]:
m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m, "OutputFlag", 0)

@variable(m, x[goods,goods], Bin)                                 
@constraint(m, c1[j in goods], sum( x[i,j] for i in goods ) == 1)      
@constraint(m, c2[i in goods], sum( x[i,j] for j in goods ) == 1) 
@constraint(m, c3[i in goods], x[i,i] == 0 )                       
@objective(m, Min, sum( x[i,j]*c[i,j] for i in goods, j in goods ))   

@variable(m, u[goods])
@constraint(m, c4[i in goods, j in goods[2:end]], u[i] - u[j] + N*x[i,j] <= N-1 )

optimize!(m)
xx = value.(x)
subtours = getAllSubtours(xx) 
display(subtours)
println("Total times: ", objective_value(m))
                                    

1-element Array{Any,1}:
 Symbol[:Donut, :Scone, :Cookie, :Bread, :CoffeeCake, :Donut]

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Total times: 37.0
